In [1]:
import os
from haystack.document_stores import ElasticsearchDocumentStore

# Get the host where Elasticsearch is running, default to localhost
host = os.environ.get("ELASTICSEARCH_HOST", "localhost")

document_store = ElasticsearchDocumentStore(
    host=host,
    username="",
    password="",
    index="document"
)

/home/dpalominop/miniconda3/envs/haystack/lib/python3.8/site-packages/espnet2/gan_tts/vits/vits.py:43: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(torch.__version__) >= LooseVersion("1.6.0"):
/home/dpalominop/miniconda3/envs/haystack/lib/python3.8/site-packages/elasticsearch/connection/base.py:190: ElasticsearchDeprecationWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchDeprecationWarning)
/home/dpalominop/miniconda3/envs/haystack/lib/python3.8/site-packages/elasticsearch/connection/http_urllib3.py:275: DeprecationWarning: HTTPResponse.getheaders() is deprecated and will be removed in urllib3 v2.1.0. Instead access HTTPResponse.headers directly.
  return response.status, respons

In [2]:
from haystack.nodes import BM25Retriever

retriever = BM25Retriever(document_store=document_store)

In [3]:
from haystack.nodes import FARMReader

reader = FARMReader(model_name_or_path="deepset/roberta-base-squad2", use_gpu=True)

In [4]:
from haystack import Pipeline

querying_pipeline = Pipeline()
querying_pipeline.add_node(component=retriever, name="Retriever", inputs=["Query"])
querying_pipeline.add_node(component=reader, name="Reader", inputs=["Retriever"])

In [5]:
prediction = querying_pipeline.run(
    query="What is BERT?",
    params={
        "Retriever": {"top_k": 10},
        "Reader": {"top_k": 5}
    }
)

Inferencing Samples:   0%|          | 0/1 [00:00<?, ? Batches/s]

In [6]:
from pprint import pprint

pprint(prediction)

{'answers': [<Answer {'answer': 'conceptually simple and empirically\npowerful', 'type': 'extractive', 'score': 0.8382189869880676, 'context': 'l task-\nspecific architecture modifications.\nBERT is conceptually simple and empirically\npowerful. It obtains new state-of-the-art re-\nsults on eleven', 'offsets_in_document': [{'start': 893, 'end': 937}], 'offsets_in_context': [{'start': 53, 'end': 97}], 'document_id': 'c6a0b7e9bce962ba6d411032d8fde7ab', 'meta': {'_split_id': 0}}>,
             <Answer {'answer': 'fine-\ntuning approaches', 'type': 'extractive', 'score': 0.6779805421829224, 'context': 'tion to the architec-\nture differences, BERT and OpenAI GPT are fine-\ntuning approaches, while ELMo is a feature-based\napproach.\nThe most comparable e', 'offsets_in_document': [{'start': 326, 'end': 349}], 'offsets_in_context': [{'start': 64, 'end': 87}], 'document_id': 'b6d499fd36829a403510593004877b03', 'meta': {'_split_id': 57}}>,
             <Answer {'answer': 'bidirectional self-at

In [7]:
from haystack.utils import print_answers

print_answers(
    prediction,
    details="minimum" ## Choose from `minimum`, `medium` and `all`
)


Query: What is BERT?
Answers:
[   {   'answer': 'conceptually simple and empirically\npowerful',
        'context': 'l task-\n'
                   'specific architecture modifications.\n'
                   'BERT is conceptually simple and empirically\n'
                   'powerful. It obtains new state-of-the-art re-\n'
                   'sults on eleven'},
    {   'answer': 'fine-\ntuning approaches',
        'context': 'tion to the architec-\n'
                   'ture differences, BERT and OpenAI GPT are fine-\n'
                   'tuning approaches, while ELMo is a feature-based\n'
                   'approach.\n'
                   'The most comparable e'},
    {   'answer': 'bidirectional self-attention',
        'context': 'Critically, however, the BERT Transformer uses\n'
                   'bidirectional self-attention, while the GPT Trans-\n'
                   'former uses constrained self-attention where every\n'
                   't'},
    {   'answer': 'the first fi